In [37]:
from pathlib import Path
import pandas as pd
import json
import re

In [44]:
training_path = Path('../data/training.csv')
chatgpt_results_path = Path('../data/chatgpt/results')
output_path = Path('../data/training_with_chatgpt.csv')

In [39]:
training_data = pd.read_csv(training_path)
training_data['chatgpt_raw_response'] = ''
training_data['chatgpt_extracted_location'] = ''
training_data['chatgpt_extracted_situation'] = ''
training_data['chatgpt_extracted_risk'] = ''
training_data['chatgpt_extracted_message'] = ''
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124248 entries, 0 to 124247
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   town                         124248 non-null  object
 1   section                      124248 non-null  int64 
 2   scenario                     124248 non-null  object
 3   part                         124248 non-null  int64 
 4   objects                      124248 non-null  object
 5   controls                     124248 non-null  object
 6   chatgpt_raw_response         124248 non-null  object
 7   chatgpt_extracted_location   124248 non-null  object
 8   chatgpt_extracted_situation  124248 non-null  object
 9   chatgpt_extracted_risk       124248 non-null  object
 10  chatgpt_extracted_message    124248 non-null  object
dtypes: int64(2), object(9)
memory usage: 10.4+ MB


In [40]:
chatgpt_results_files = list(chatgpt_results_path.glob('*.txt'))
chatgpt_results_files.sort(key=lambda x: int(x.stem))

In [41]:
start_json_pattern = r'```(json)*(\n)*'

In [42]:
for index, row in training_data.iterrows():
    with open(chatgpt_results_files[index], 'r') as f:
        chatgpt_raw_response = f.read()
        training_data.at[index, 'chatgpt_raw_response'] = chatgpt_raw_response
        try:
            chatgpt_raw_response = re.sub(start_json_pattern, '', chatgpt_raw_response).strip()
            chatgpt_response = json.loads(chatgpt_raw_response)
        except json.JSONDecodeError:
            print(chatgpt_raw_response)
        training_data.at[index, 'chatgpt_extracted_location'] = chatgpt_response['location']
        training_data.at[index, 'chatgpt_extracted_situation'] = chatgpt_response['situation']
        training_data.at[index, 'chatgpt_extracted_risk'] = chatgpt_response['risk']
        training_data.at[index, 'chatgpt_extracted_message'] = chatgpt_response['message']

In [43]:
training_data.head(5)

,town,section,scenario,part,objects,controls,chatgpt_raw_response,chatgpt_extracted_location,chatgpt_extracted_situation,chatgpt_extracted_risk,chatgpt_extracted_message
0,Town12,81,StaticCutIn,1,"[(30, 'vehicle.ford.mustang'), (15, 'vehicle.m...","[{'throttle': 0.75, 'steer': 2.133598718501161...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching multiple vehicles,medium,"Stay focused, be aware of surrounding vehicles..."
1,Town12,81,StaticCutIn,1,"[(29, 'vehicle.ford.mustang'), (14, 'vehicle.m...","[{'throttle': 0.75, 'steer': 0.000891609524842...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching multiple vehicles and a traffic light,medium,"Stay alert, approaching vehicles and traffic l..."
2,Town12,81,StaticCutIn,1,"[(28, 'vehicle.ford.mustang'), (13, 'vehicle.m...","[{'throttle': 0.75, 'steer': -0.00017667813517...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching multiple vehicles and a traffic light,medium,"Stay focused, be prepared to slow down for the..."
3,Town12,81,StaticCutIn,1,"[(30, 'vehicle.nissan.patrol_2021'), (30, 'veh...","[{'throttle': 0.0, 'steer': 0.0002305144735146...","{\n ""location"": ""Urban area"",\n ""situation"":...",Urban area,Approaching multiple vehicles and a stop sign,medium,"Stay alert, slow down, and prepare to stop at ..."
4,Town12,81,StaticCutIn,1,"[(48, 'vehicle.mini.cooper_s_2021'), (7, 'vehi...","[{'throttle': 0.0, 'steer': -0.000179785565705...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching a stop sign with vehicles nearby,medium,"Approach the stop sign cautiously, there are v..."


In [46]:
training_data.to_csv(output_path, index=False)